In [ ]:
!pip install transformers torch pandas pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.5 MB/s eta 0:00:00


In [ ]:
!pip install autoawq

INFO: pip is looking at multiple versions of autoawq-kernels to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/19

In [ ]:
# Importing the dependencies
import pandas as pd
import pdfplumber
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
# Loading the dataset
data_path = '/content/data.csv'
df = pd.read_csv(data_path)

In [ ]:
# Loading the Qwen model and tokenizer
model_name = "Qwen/Qwen2-1.5B-Instruct-AWQ"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to('cuda')

`low_cpu_mem_usage` was None, now set to True since model is quantized.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
# Function to extract text from resume PDF
def extract_resume_text(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [ ]:
import re
from datetime import datetime

In [ ]:
# Function to query the model for years of experience
def ask_years_of_experience(resume_text):
    question = "Find the number of years of experience the applicant have. Answer should be in only in one floating point number?"
    inputs = tokenizer(f"Question: {question}\nResume: {resume_text}", return_tensors="pt")
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Function to extract date ranges from resume text
def extract_date_ranges(resume_text):
    date_pattern = r'(\d{1,2}/\d{1,2}/\d{2,4})\s*-\s*(\d{1,2}/\d{1,2}/\d{2,4})'
    matches = re.findall(date_pattern, resume_text)
    date_ranges = []
    for start, end in matches:
        try:
            start_date = datetime.strptime(start, '%m/%d/%Y')
            end_date = datetime.strptime(end, '%m/%d/%Y')
            date_ranges.append((start_date, end_date))
        except ValueError:
            # Handle parsing errors
            continue

    return date_ranges

# Function to check for overlapping date ranges
def check_overlapping_dates(resume_text):
    date_ranges = extract_date_ranges(resume_text)
    overlaps = []

    for i in range(len(date_ranges)):
        for j in range(i + 1, len(date_ranges)):
            # Check if ranges overlap
            start_a, end_a = date_ranges[i]
            start_b, end_b = date_ranges[j]
            if start_a <= end_b and start_b <= end_a:  # Overlap condition
                overlaps.append((date_ranges[i], date_ranges[j]))

    return overlaps

# Function to ask for years of experience with additional context from overlapping dates
def ask_years_of_experience_with_overlap(resume_text):
    question = "Find the number of years of experience the applicant has. Answer should be in only one floating point number."
    overlaps = check_overlapping_dates(resume_text)
    overlap_info = f"Overlapping date ranges found: {overlaps}" if overlaps else "No overlapping date ranges found."

    inputs = tokenizer(f"Question: {question}\nResume: {resume_text}\n{overlap_info}", return_tensors="pt")
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer


In [ ]:
# Create a new column for years of experience and process resumes
df['Years_of_Experience'] = ""

In [ ]:
# Iterate through each row in the dataset
for index, row in df.iterrows():
    applicant_id = row['ID']
    resume_file = f"/content/Resume_of_ID_{applicant_id}.pdf"  # Resume path

    # Extract text from resume
    resume_text = extract_resume_text(resume_file)

    # Get the answer from the model
    experience_answer = ask_years_of_experience(resume_text)

    # Save the answer in the dataframe
    df.at[index, 'Years_of_Experience'] = experience_answer

    # Checkpoint: Print status and save intermediate file every 10 resumes
    if index % 10 == 0:
        print(f"Processed {index+1} resumes, current ID: {applicant_id}, extracted experience: {experience_answer}")
        df.to_csv("/content/processed_experience_data_checkpoint.csv", index=False)

Processed 11 resumes, current ID: 10, extracted experience: Question: Find the number of years of experience the applicant have. Answer should be in only in one floating point number?
Resume: OPERATIONS MANAGER
Summary
Experienced client service professional accustomed to managing teams functioning in a fast-paced environment in support of multiple products
across various business units. Solid background in building strong and effective teams
Highlights
KYC, Anti Money Laundering, and legal documentations
Excellent organizational and time management skills
Strong interpersonal and communication skills at all levels Microsoft Office (Excel, Word, PowerPoint, SharePoint, and Project)
Employee relations (hiring, terminations, performance management, development)
SEC Licenses Series 6 and 63 (Expired. Willing to retake.)
Proficiency in Adobe products (Photoshop, Lightroom, Illustrator)
Spanish fluency, oral and written
Experience
02/2010 to 08/2014
Operations Manager Company Name ï¼​ City 

KeyboardInterrupt: 

In [ ]:
df

,ID,Recommenders ID,Years_of_Experience
0,0,"[218, 391]",Question: Find the number of years of experien...
1,1,"[412, 869, 233, 289]",Question: Find the number of years of experien...
2,2,"[582, 624, 592, 662, 469]",Question: Find the number of years of experien...
3,3,"[194, 122]",Question: Find the number of years of experien...
4,4,"[763, 726, 589, 977, 950, 543, 30]",Question: Find the number of years of experien...
...,...,...,...
959,995,"[51, 290, 370]",
960,996,"[254, 890, 296]",
961,997,"[850, 228, 194]",
962,998,"[258, 471]",


In [ ]:

# Save the final dataframe with years of experience
df.to_csv("/mnt/data/final_experience_data.csv", index=False)

print("Processing complete and saved to final_experience_data.csv")
